In [ ]:
import pandas as pd


In [ ]:
full_df = pd.read_csv('/content/output1.csv')

In [ ]:
full_df.head(2)

,ID,Text,generated_summary,Original_summary,Rouge_score_Uni_gen,Rouge_score_fulllength_gen,Rouge_score_Uni_orig,Rouge_score_fulllength_orig,rating
0,1,"MADISON, Wis. -- More than 700 million adults ...",zapping the stomach to reduce appetite could b...,UW-Madison graduate student Guang Yao (left) a...,0.328767,0.124533,0.234637,0.128492,2
1,2,"Wilmington, DE, December 17, 2018 - The CRISPR...",gene-editing systems are already used in cance...,This news release reports on preliminary resea...,0.214180,0.109306,0.097600,0.070400,3


In [ ]:
df = full_df[['Original_summary','rating']]
df.head()

,Original_summary,rating
0,UW-Madison graduate student Guang Yao (left) a...,2
1,This news release reports on preliminary resea...,3
2,This news release focuses on a study of the gu...,1
3,This news release summarizes findings from a r...,1
4,The release focuses on two papers that discuss...,3


In [ ]:
import numpy as np
import torch
import random
def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
seed_all(202020)

In [ ]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    # if is_torch_available():
    import torch

    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # ^^ safe to call this function even if cuda is not available
    # if is_tf_available():
    import tensorflow as tf

    tf.random.set_seed(seed)
set_seed(202020)

In [ ]:
df.head()

,Original_summary,rating
0,UW-Madison graduate student Guang Yao (left) a...,2
1,This news release reports on preliminary resea...,3
2,This news release focuses on a study of the gu...,1
3,This news release summarizes findings from a r...,1
4,The release focuses on two papers that discuss...,3


In [ ]:
df['rating'] = df['rating'].map({0:0,1:0,2:0,3:1,4:1})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df['Original_summary'] = df['Original_summary'].astype(str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df1 = train_test_split(df, test_size=0.1,random_state=202020)

print('train shape: ',train_df.shape)
print('test shape: ',test_df1.shape)

train shape:  (270, 2)
test shape:  (30, 2)


In [ ]:
# !pip install simpletransformers


# # check installed version
# !pip freeze | grep simpletransformers
# # simpletransformers==0.28.2

In [ ]:
from simpletransformers.classification import ClassificationModel

model = ClassificationModel("roberta", "roberta-base", use_cuda=True,num_labels=2, args={'train_batch_size':4,
                                                                          'eval_batch_size':4,
                                                                         'reprocess_input_data': True,
                                                                         'overwrite_output_dir': True,
                                                                         'fp16': False,
                                                                         'do_lower_case': False,
                                                                         'num_train_epochs': 4,
                                                                         'max_seq_length': 32,
                                                                         'regression': False,
                                                                         'manual_seed': 202020,
                                                                         "learning_rate":1e-5,
                                                                         'weight_decay':1e-5,
                                                                         "save_eval_checkpoints": False,
                                                                         "save_model_every_epoch": False,
                                                                         "silent": False})


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [ ]:
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:387: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


(272, 0.6532994011307464)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score



def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)

result

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:966: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


{'acc': 0.6,
 'eval_loss': 0.7161753848195076,
 'f1': 0.6,
 'fn': 5,
 'fp': 7,
 'mcc': 0.20535714285714285,
 'tn': 9,
 'tp': 9}

In [ ]:
from scipy.special import softmax

raw_outputs_val = model.eval_model(test_df1)[1]
raw_outputs_val = softmax(raw_outputs_val,axis=1)[:,1]


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:966: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


In [ ]:
outdf = pd.DataFrame(raw_outputs_val,columns=['Probabilities'])
outdf['predicted'] = np.where(outdf['Probabilities'] >0.45,1,0)
outdf['actual'] = test_df1['rating'].values

In [ ]:
from sklearn.metrics import *

In [ ]:
confusion_matrix(outdf['actual'].values,outdf['predicted'].values)

array([[ 9,  7],
       [ 3, 11]])

In [ ]:
f1_score(outdf['actual'].values,outdf['predicted'].values)

0.6875000000000001

In [ ]:
precision_score(outdf['actual'].values,outdf['predicted'].values)

0.6111111111111112

In [ ]:
recall_score(outdf['actual'].values,outdf['predicted'].values)

0.7857142857142857

In [ ]:
outdf.head(30)

,Probabilities,predicted,actual
0,0.434685,0,0
1,0.228214,0,1
2,0.744320,1,1
3,0.504036,1,1
4,0.831428,1,0
5,0.502102,1,1
6,0.758567,1,0
7,0.469503,0,1
8,0.354564,0,0
9,0.589797,1,1
